<h1><center>FEDERATED LEARNING MODEL</center></h1>

<h3>IMPORTING LIBARIES</h3>

In [13]:

import os
from imutils import paths

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf

import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K


<h3>Preparing DataSet</h3>

<p>Step 1 : Extracting Images from the  Folders</p>
<p>Step 2 : Adjusting The angles</p>
<p>Step 3 : Normalizing the Image</p>

In [14]:

TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Test"

train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)
test_dataset = image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')


validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')



#split data into training and test set
X_train,Y_train = next(train_generator)
X_test,Y_test = next(validation_generator)

Found 224 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


<h4>Creating Clients</h4>

In [15]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    client_names = ['{}_{}'.format( initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))}

clients = create_clients(X_train, Y_train, num_clients=10, initial='client')

In [16]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [17]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(len(Y_test))

<h3>Creating CNN Model </h3>

In [18]:
class SimpleMLP:
    @staticmethod
    def build():
        model = Sequential()
        model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
        model.add(Conv2D(64,(3,3),activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(64,(3,3),activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(128,(3,3),activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(64,activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1,activation='sigmoid'))
        return model

In [19]:
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               ) 

/home/hashiraflame/.local/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


<h3>Calculating the Weights for each Clients</h3>

In [20]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [21]:
def predict(X_test,model):
    pred_img = model.predict(X_test)
    print(np.argmax(pred_img))


<h3>Calculating the weights of the Global Model for each Clients</h3>

In [22]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build()
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build()
        local_model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])
        callback = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)]
        local_model.fit(train_generator,epochs=10,validation_data=validation_generator,callbacks=[callback])   
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

Epoch 1/10
7/7 [==============================] - 240s 38s/step - loss: 2.2184 - accuracy: 0.5402 - val_loss: 0.6867 - val_accuracy: 0.9167
Epoch 2/10
7/7 [==============================] - 54s 7s/step - loss: 0.6601 - accuracy: 0.6339 - val_loss: 0.6479 - val_accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - 61s 9s/step - loss: 0.5779 - accuracy: 0.6875 - val_loss: 0.4662 - val_accuracy: 0.8667
Epoch 4/10
7/7 [==============================] - 75s 11s/step - loss: 0.3415 - accuracy: 0.8616 - val_loss: 0.2311 - val_accuracy: 0.9333
Epoch 5/10
7/7 [==============================] - 45s 6s/step - loss: 0.4006 - accuracy: 0.8393 - val_loss: 0.3930 - val_accuracy: 0.9667
Epoch 6/10
7/7 [==============================] - 54s 8s/step - loss: 0.3274 - accuracy: 0.8571 - val_loss: 0.1812 - val_accuracy: 0.9667
Epoch 7/10
7/7 [==============================] - 44s 6s/step - loss: 0.3932 - accuracy: 0.8170 - val_loss: 0.3436 - val_accuracy: 0.9667
Epoch 8/10
7/7 [===============

In [ ]:
test_model(X_test,Y_test,global_model,100)